In [236]:
import pandas as pd
from codes.db import * 
from codes.lib import *
from codes.riverlog_for_gis import *
import pandasql as ps

gd={}
conn=connect_db()
if 1:
    load_ods(gd)
    riverlog_info_setup(gd)
#sql_keys
#sql_keypairs

Connected to the PostgreSQL database...
sys 的 sheets:
 dict_keys(['table_def', 's_table_desc', 's_syspar', 's_info_line', 's_info_area', 's_timeseq_rpt', 's_repeat_job', 's_topology_kind', 's_gis_layer', 's_info_point', 's_topology_transfer', 's_topology_node', 's_topology_edge']) 
basic 的 sheets:
 dict_keys(['table_def', 'b_表單說明', 'b_colmeta', 'b_水資源分區', 'b_水資源局', 'b_河川局', 'b_流域', 'b_河川', 'b_排水', 'b_水庫', 'b_水質水量保護區', 'b_水庫集水範圍', 'b_水庫集水區敏感區', 'b_堤防', 'b_排水設施', 'b_水門', 'b_抽水站', 'b_河川斷面樁', 'b_雨量站', 'b_河川水位站', 'b_浮標站', 'b_潮位站', 'b_地下水分區', 'b_地下水觀測井']) 
rain-station: output/rain-station.csv saved, shape = (2149, 6)
reservoir-info: output/reservoir-info.csv saved, shape = (152, 22)
waterLevel-station: output/waterLevel-station.csv saved, shape = (5176, 13)
waterLevelDrain-station: output/waterLevelDrain-station.csv saved, shape = (1189, 4)
waterLevelAgri-station: output/waterLevelAgri-station.csv saved, shape = (359, 4)
sewer-station: output/sewer-station.csv saved, shape = (310, 8)
tide-s

# 使用範例

In [2]:
#df = sql_to_df(conn,sql)  
#sql_exec(conn,sql)
#print(view_by_key(None,None).keys())
#df=view_by_key(conn,'info_tables')
#key_to_value(conn,'b_流域','basin_id',130000,"basin_cname")
#key_to_value_help(conn,'b_流域','basin_id') #key_to_value_help(conn,None,None)
#df=keypar_to_view('basin_test',['頭前溪','濁水溪','大甲溪'],"str")
#df=keypar_to_view('basin_test2',['頭前溪','濁水溪'],"pos")
#df=keypar_to_view(conn,'station_by_basinname',['頭前溪'],"pos")
#df

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,postgis,public,b_地下水分區,BASE TABLE,None,None,None,None,None,YES,NO,None
1,postgis,public,b_地下水觀測井,BASE TABLE,None,None,None,None,None,YES,NO,None
2,postgis,public,b_堤防,BASE TABLE,None,None,None,None,None,YES,NO,None
3,postgis,public,b_抽水站,BASE TABLE,None,None,None,None,None,YES,NO,None
4,postgis,public,b_排水,BASE TABLE,None,None,None,None,None,YES,NO,None
...,...,...,...,...,...,...,...,...,...,...,...,...
128,postgis,public,watrelin,BASE TABLE,None,None,None,None,None,YES,NO,None
129,postgis,public,wrarb,BASE TABLE,None,None,None,None,None,YES,NO,None
130,postgis,public,yl106107,BASE TABLE,None,None,None,None,None,YES,NO,None
131,postgis,public,yl81108_polyline,BASE TABLE,None,None,None,None,None,YES,NO,None


In [261]:
if 0:
    #sql = "select * from rivercode order by river_id"
    sql = "select ST_AsGeoJson(geom) as geom_json from basin where basin_name='頭前溪'"
    df = sql_to_df(conn,sql)  

if 0:
    sql="""
    CREATE TABLE "河川局" (
        a DECIMAL, 
        rvb_no DECIMAL, 
        rvb_name VARCHAR, 
        area BOOLEAN, 
        "水資源分區代號" BOOLEAN
    );

    """
    sql_exec(conn,sql)

if 0:
    close_db(conn)
if 0:
    #print("\n".join(get_table_list()))
    df = view_by_key(conn,'b_流域')
    

if 0:
    sql="select * from r_rain_station where \"stationID\"='00H710'"
    df = sql_to_df(conn,sql)
#df

if 0: #同名欄位分析
    df = view_by_key(conn,'info_columns')
    value_bins = df['column_name'].value_counts()
    #value_bins[value_bins>1].to_csv('output/cols_count.csv')
    value_bins[value_bins>10]
    #df[df['column_name']=='gid']

if 0: # 當場加 sql_keys 來使用
    sql_keys['level_riverpoly_rivercode']=\
    "select S.st_no,S.name_c,S.tri_name,ST_Distance(S.geom,R.geom) as dist from RIVWLSTA_e as S \
        join riverpoly_rivercode as R\
            on S.tri_name=R.river_name\
    order by dist;"
    df=view_by_key(conn,'level_riverpoly_rivercode')
if 0: # 當場加 sql_keypairs 來使用
    sql_keypairs['level_in_reservoir']=\
    "select S.st_no, S.name_c, R.name from RIVWLSTA_e as S \
        join reservoir as R \
            on ST_Within(S.geom,R.geom) \
    where name in (%s);"
    df=keypar_to_view(conn,'level_in_reservoir',['隆恩堰'],"str")

if 0: # save geom_json to file
    with open('output/test2.geojson', 'w') as f:
        f.write(df['geom_json'].to_list()[0])


In [260]:

#df1=df['geom_json']
#pd.savetxt('output/test2.geojson', df1.values)



# 目前練習

In [15]:
#key_to_value_help(conn,None,None)
#key_to_value_help(conn,'b_河川','river_id')
#key_to_value_help(conn,'rivercode',None)

df=view_by_key(conn,'rivercode')
#print(key_to_value(conn,'b_河川','river_id',130000.0,"river_cname"))

df

,river_id,river_name,river_ename,basin_id,in_id,gov_id,river_link
0,165000,鹽水溪,Yanshui River,1650.0,0,6.0,0@165000
1,165040,大洲排水,Dazhou Drainage,1650.0,165000,6.0,0@165000@165040
2,165050,柴頭港溪排水,Chaitougang River Drainage,1650.0,165000,6.0,0@165000@165050
3,165060,鹽水溪排水,Yanshui River Drainage,1650.0,165000,6.0,0@165000@165060
4,165061,曾文溪排水,Zengwen River Drainage,1650.0,165060,6.0,0@165000@165060@165061
...,...,...,...,...,...,...,...
522,14306A,四塊厝圳支線,Si Kuai Cuo Zun Ahi Zhi Xian,1430.0,143061,3.0,0@143000@143060@143061@14306A
523,14306C,員寶庄圳支線,Yuan Bao Zhuang Zun Zhi Xian,1430.0,143061,3.0,0@143000@143060@143061@14306C
524,14306D,大埔厝圳支線,Da Pu Cuo Zun Zhi Xian,1430.0,143061,3.0,0@143000@143060@143061@14306D
525,15402A,興安中排四,Xing An Zhong Pai Si,1540.0,154023,5.0,0@154000@154020@154023@15402A


In [38]:
# 測站離河流距離
sql="select tri_name,st_no,name_c,ST_AsEWKT(geom) as ewkt from RIVWLSTA_e \
    where tri_name in ('頭前溪','上坪溪','油羅溪');"
#sql="select tri_name,st_no,name_c,ST_AsEWKT(geom) as ewkt from RIVWLSTA_e;"
df = sql_to_df(conn,sql)

print("水位站(RIVWLSTA_e) 跟河川河道(riverpoly)的距離:")
print("測站,河流,距離")
for row in df.iterrows():
    sql = "select river_name,ST_Distance(geom,ST_GeomFromText('%s')) as dist from riverpoly \
        where river_name='%s' order by dist limit  1" \
        % (row[1]['ewkt'],row[1]['tri_name'])
    df1 = sql_to_df(conn,sql)
    dist = df1['dist'].to_list()
    if len(dist)==0:
        print("%s,%s can't get distance" %(row[1]['name_c'],row[1]['tri_name']))
    else:
        if dist[0]>=0: #距離 filter
            print("%s,%s,%.2f" %(row[1]['name_c'],row[1]['tri_name'],dist[0]))

水位站(RIVWLSTA_e) 跟河川河道(riverpoly)的距離:
測站,河流,距離
內灣,油羅溪,0.00
上坪,上坪溪,0.00
竹林大橋,頭前溪,0.00
經國橋,頭前溪,0.00
竹林大橋(左岸),頭前溪,0.00
五峰大橋,上坪溪,38.12
舊港橋,頭前溪,164.20
竹港大橋,頭前溪,18.38
中正大橋,頭前溪,44.94
頭前溪橋,頭前溪,1.64
道將圳,頭前溪,7764.45


In [ ]:
sql="select tri_name,st_no,name_c,ST_AsEWKT(geom) as ewkt from RIVWLSTA_e;"
df = sql_to_df(conn,sql)

print("水位站(RIVWLSTA_e) 跟河川河道(riverpoly)的距離:")
print("測站,河流,距離")
for row in df.iterrows():
    sql = "select river_name,ST_Distance(geom,ST_GeomFromText('%s')) as dist from riverpoly \
        order by dist limit  1" \
        % (row[1]['ewkt'])
    print("sql=%s" % (sql))
    df1 = sql_to_df(conn,sql)
    dist = df1['dist'].to_list()
    if len(dist)==0:
        print("%s,%s can't get distance" %(row[1]['name_c'],row[1]['tri_name']))
    else:
        if dist[0]>=500: #距離 filter
            print("%s,%s,%.2f" %(row[1]['name_c'],row[1]['tri_name'],dist[0]))

# 拓墣

In [242]:
%matplotlib notebook
import matplotlib.pyplot as plt
import networkx as nx
def gen_rivertree(conn,filename, rivercode="",id_with_name=False):
    """ 
        load river tree from db:rivercode to networkx graph,df, output dot format
        parameter: 
            filename: filename for dot
            rivercode: rivercode or "" for all
            id_with_name: graph ID with name?
            
        ex: gen_rivertree(dot_filename, "130000")
        return: [G,df]
    
    """

    G = nx.Graph()
    try:
        sql = "select * from rivercode order by river_id"
        df = pd.read_sql(sql, conn)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

    
    lines = []
    dot_strs = {}
    for index, row in df.iterrows():
        link = row['river_link']
        if not rivercode=="":
            if not rivercode in link:
                continue
        #print("processing: %s" %(link))
        cols = link.split("@")
        for i in range(len(cols)-1):
            if cols[i]=='0':
                river_name1 = "海"
                name1 = river_name1
            else:
                river_name1 = df[df['river_id']==cols[i]]['river_name'].values.tolist()
                if len(river_name1)>0:
                    name1 = river_name1[0]
                else:
                    name1 = "Undefined"
            river_name2 = df[df['river_id']==cols[i+1]]['river_name'].values.tolist()
            
            #print("river_name2=%s" %(river_name2[0]))
            if len(river_name2)>0:
                name2 = river_name2[0]
            else:
                name2 = "Undefined"
            dot_str = "\"%s_\\n%s\"->\"%s_\\n%s\"" %(cols[i+1],name2,cols[i],name1)
            #G.add_nodes_from([cols[i], cols[i+1]])
            if id_with_name:
                n2="%s\n%s" %(cols[i+1],name2)
                n1="%s\n%s" %(cols[i],name1)
                G.add_node(n2, name=name2)
                G.add_node(n1, name=name1)
                G.add_edges_from([(n2, n1)])
            else:
                G.add_node(cols[i+1], name=name2)
                G.add_node(cols[i], name=name1)
                G.add_edges_from([(cols[i+1], cols[i])])
            if dot_str not in dot_strs:
                dot_strs[dot_str]=1
                #print("dot string: %s" %(dot_str))
                lines.append(dot_str)                     
       
    fp = open(filename, "w")
    lines.sort()
    fp.write("%s%s%s" %("digraph G {\n\t\n","\t" + "\n\t".join(lines),"\n}\n"))
    fp.close() 
    return G,df



def output_graph(G,labels,output_id):
    """
    output_id: 1(print) , 2(gml), 3(plot)
    """
    if output_id==1: #print
        print(list(G.nodes))
        print(list(G.edges))
        #print(G.nodes['130000'])
    if output_id==2: #gml
        nx.readwrite.gml.write_gml(G,"output/rivertree.gml")
    if output_id==3: #plot
        from matplotlib.font_manager import FontProperties
        myfont = FontProperties(fname=r'/Library/Fonts/Microsoft/SimSun.ttf')
        pos = nx.spring_layout(G)
        colors = range(len(G.edges))
        options = {
            "node_color": "#A0CBE2",
            #"edge_color": colors,
            "width": 1,
            #"edge_cmap": "#A0CBE2",
            "with_labels": True,
        }
        if not labels is None:
            nx.draw(G, pos, font_family='SimSun',labels=labels,**options)
        else:
            nx.draw(G, pos, font_family='SimSun',**options)
        #fig = plt.gcf()
        #fig.set_size_inches(8.5, 6)
        plt.show()
    if output_id==4: # write dot,need pygraphviz (too many packages need to install, abort)
        from networkx.drawing.nx_agraph import write_dot
        write_dot(G, "output/topology.dot")

    #nx.readwrite.nx_shp.write_shp(G, "output/rivertree.shp")

def gen_topology(filename,id_with_name):
    """
        load topoloy from gd['sys']['s_topology_node'],gd['sys']['s_topology_edge'] 
            to networkx graph,df, output dot format
        parameter: 
            filename: filename for dot
            
        ex: gen_topology(dot_filename)
        return: [G,labels]
    """
    G = nx.Graph()
    df_node = gd['sys']['s_topology_node'] #node_id,node_name
    df_edge = gd['sys']['s_topology_edge'] #edge_id,node1_id,node2_id,edge_name
    
    lines = []
    dot_strs = {}
    labels = {}
    for index, row in df_node.iterrows():

        labels[row['node_id']]=row['node_name']
        G.add_node(row['node_id'],node_name=row['node_name'],kind=row['kind'],\
            layer_name=row['layer_name'],feature_def=row['feature_def'])

    for index, row in df_edge.iterrows():

        n1=row['node1_id']
        n2=row['node2_id']
        #G.add_edges_from([(n1, n2)],\
        #    edge_id=row['edge_id'],edge_name=row['edge_name'],kind=row['kind'])
        G.add_weighted_edges_from([(n1, n2,1)],\
            edge_id=row['edge_id'],edge_name=row['edge_name'],kind=row['kind'])
        
        #G.add_edges_from(n1, n2)

        t1=get_value_by_index(df_node,"node_id=%s" %(n1), 'node_name')
        t2=get_value_by_index(df_node,"node_id=%s" %(n2), 'node_name')
        if id_with_name: 
            dot_str = "\"%s_\\n%s\"->\"%s_\\n%s\"  [dir=none]" %(n1,t1,n2,t2)
        else:
            dot_str = "\"%s\"->\"%s\" [dir=none]" %(t1,t2)
        if dot_str not in dot_strs:
            dot_strs[dot_str]=1
            #print("dot string: %s" %(dot_str))
            lines.append(dot_str)                        
    if 1:
        fp = open(filename, "w")
        lines.sort()
        fp.write("%s%s%s" %("digraph G {\n\t\n","\t" + "\n\t".join(lines),"\n}\n"))
        fp.close() 
    return G,labels
def topo_node_desc(G,node_id):
    import numpy as np
    if not node_id in G._node.keys():
        print("%s not exist!" %(node_id))

    df_node = gd['sys']['s_topology_node'] #node_id,node_name
    df_edge = gd['sys']['s_topology_edge'] #edge_id,node1_id,node2_id,edge_name
    df_tran = gd['sys']['s_topology_transfer'] #from_類別	類別情境與問題	to_類別	json_def	說明
    
    #node
    node = G._node[node_id]
    #node_name=get_value_by_index(df_node,"node_id=%s" %(node_id), 'node_name')
    node_name = node['node_name']
    print("node %s(%s):%s have these edges:" %(node_id,node_name,node['kind']))
    #edge
    for edge_id in G._adj[node_id].keys():
        #edge_name=get_value_by_index(df_node,"node_id=%s" %(edge), 'node_name')
        edge = G._adj[node_id][edge_id]
        edge_name = edge['edge_name']
        print("%s(%s) -- %s(%s):%s" %(node_id,node_name,edge_id,edge_name,edge['kind']))
    
    #transfer
    f_tran=df_tran['from_類別']==node['kind']
    print("%s(%s) 有以下問題：" %(node_id,node_name))
    for index, row in df_tran[f_tran].iterrows():
        print("%s" %(row['類別情境與問題']))
        #if not np.isnan( row['json_def']):
        if not pd.isnull(row['json_def']):
            print("json_def:%s" %(row['json_def']))
            
    # 圖層圖位與內容
    layer_name = node['layer_name']
    print("%s(%s) locate at layer_name %s, feature %s" %(node_id,node_name,layer_name,node['feature_def'])) 
    
    # 圖位中的資訊
    if layer_name in gd['sys'].keys():
        df_layer=gd['sys'][layer_name]
        desc=get_value_by_index(df_layer,node['feature_def'], 'desc')
        print("feature description: %s" %(desc))
        
    
    
            
def topo_question_act(G,node_id,q_str,q_type, q_par=None):
    """ 某個 node 點選，某個問題，目前指定動作跟參數
    q_str: 類別情境與問題
    q_type:: kind: 找某一類別的 node
    q_par: 可選參數，是個 list,預設為 None
    """
    df_tran = gd['sys']['s_topology_transfer']
    df_node = gd['sys']['s_topology_node'] #node_id,node_name
    f_q=df_tran['類別情境與問題']==q_str
    df_tran1=df_tran[f_q]
    node= G._node[node_id]
    node_name = node['node_name']
    
    if q_type=="kind": #找某一類別的 node
        for n2_id in G._adj[node_id].keys():
            edge = G._adj[node_id][n2_id]
            edge_name = edge['edge_name']
            #f_node=df_node['node_id']==n2_id
            node2= G._node[n2_id]
            kind = node2['kind']
            #node_name2=df_node[f_node]['node_name'].to_list()[0]
            node_name2 = node2['node_name']
            if kind==df_tran1['to_類別'].to_list()[0]:
                print("%s(%s) 問 %s 答案：  %s(%s):%s" %(node_id,node_name,q_str,n2_id,node_name2,kind))
                return n2_id
            else:
                pass
                #print("INFO: checking edge: %s" %(edge_name))

def topo_set_bylen(G,node_id,length):
    """ 取得跟某點距離<=某值的點位
    """
    from networkx.algorithms import shortest_paths as sp
    import networkx.classes.function as fun
    print("shortest_path_length of %s:" %(node_id))
    paths=sp.shortest_path_length(G,source=node_id)
    node_list=[]
    edge_key=[]
    for p in paths:
        if paths[p]!=0:
            if paths[p]<=length:
                print("with %s length %s" %(p,paths[p]))
                node_list.append(p)
        if paths[p]<length:
            edge_key.append(p)
    edge_list=fun.edges(G,edge_key)
    
    return node_list,edge_list

def topo_check(G):
    import networkx.classes.function as fun
    #沒有 edge 的點
    degree = fun.degree(G)
    print("these nodes have no edge:")
    for d in degree:
        #print(d)
        if d[1]==0:
            print("%s" %(d[0]))
    
    

In [243]:
#gen_rivertree("") # 全部
if 0:
    [G,df]= gen_rivertree(conn,"output/rivertree.dot","130000",True) 
    output_graph(G,None)

if 1:
    [G,labels] = gen_topology("output/topology.dot",True)
    #output_graph(G,labels,3)
    #G._node['hash-3']
    #G._adj['hash-3'].keys()
    
if 0:
    topo_node_desc(G,'hash-2')

    node_id=topo_question_act(G,'hash-2','取水口為何','kind',None)
    #topo_question_act(G,'hash-3','集水區為何','kind',None)
    
    topo_node_desc(G,node_id)

if 0: # 某點距離<3 的所有node,edge
    node_id='hash-1'
    length=3
    node_list,edge_list = topo_set_bylen(G,node_id,length)
    print("%s nearby %i nodes: %s" %(node_id,length, node_list))
    print("edges:%s" %(edge_list)) 
if 1:
    topo_check(G)

these nodes have no edge:
hash-7


# 拓樸演算法學習

In [249]:
if 0:
    from networkx.algorithms import approximation as approx
    #approx.node_connectivity(G,s='hash-3',t='hash-7')
    #approx.local_node_connectivity(G,'hash-2','hash-5')

if 0:
    from networkx.algorithms import distance_measures as dm
    print("----- distance_measures:")
    print("center: %s " %(dm.center(G)))

if 0:
    from networkx.algorithms import shortest_paths as sp
    print("----- shortest_paths:")
    node_id='hash-3'
    print("shortest_path_length of %s:" %(node_id))
    paths=sp.shortest_path_length(G,source=node_id)
    for p in paths:
          print("%s:%s" %(p,paths[p]))
    print("all shortest_path:")
    paths = sp.shortest_path(G)
    for p in paths.keys():
        print("%s path:%s" %(p,paths[p]))
    print("shortest_path from %s to %s:%s" %('hash-1','hash-6',sp.shortest_path(G,source='hash-1',target='hash-6')))
    print("has_path:: %s to %s:%s" %('hash-1','hash-6', sp.has_path(G,'hash-1','hash-6')))

    
if 0: #function
    import networkx.classes.function as fun
    print("---------- function:")
    print("----- Graph:")
    print("degress: %s " %(fun.degree(G)))
    print("degree_histogram:%s" %(fun.degree_histogram(G)))
    print("density: %s " %(fun.density(G)))
    print("info: %s " %(fun.info(G)))
    print("is_directed: %s " %(fun.is_directed(G)))
    print("subgraph: %s " %(fun.subgraph(G,None)))
    
    print("----- Nodes:")
    print("nodes: %s " %(fun.nodes(G)))
    print("number_of_nodes: %s " %(fun.number_of_nodes(G)))
    print("neighbors of %s:" %('hash-3'))
    for n in fun.neighbors(G,'hash-3'):
        print(n)
    print("all_neighbors of %s " %('hash-3'))
    for n in fun.all_neighbors(G,'hash-3'):
        print(n)
    print("non_neighbors of %s " %('hash-3'))
    for n in fun.non_neighbors(G,'hash-3'):
        print(n)
    print("common_neighbors of %s -- %s " %('hash-2','hash-4'))
    for n in fun.common_neighbors(G,'hash-2','hash-4'):
        print(n)
    print("----- Edges:")
    print("all edges: %s " %(fun.edges(G)))
    nodes=['hash-3','hash-1']
    print("nodes %s edges: %s " %(nodes,fun.edges(G,nodes)))
    print("non_edges: ")
    for n in fun.non_edges(G):
        print(n)
    print("----- Attributes:")
    print("is_weighted: %s " %(fun.is_weighted(G)))
    print("get_node_attributes of kind:%s " %(fun.get_node_attributes(G,'kind')))
    path=['hash-2','hash-3','hash-4']
    print("is_path %s: %s " %(path,fun.is_path(G,path)))
    
    
    
    #fun.nodes(G)
    #fun.edges(G)


----- shortest_paths:
shortest_path_length of hash-3:
hash-3:0
hash-6:1
hash-5:1
hash-2:1
hash-4:1
hash-1:2
all shortest_path:
hash-1 path:{'hash-1': ['hash-1'], 'hash-2': ['hash-1', 'hash-2'], 'hash-3': ['hash-1', 'hash-2', 'hash-3'], 'hash-4': ['hash-1', 'hash-2', 'hash-3', 'hash-4'], 'hash-5': ['hash-1', 'hash-2', 'hash-3', 'hash-5'], 'hash-6': ['hash-1', 'hash-2', 'hash-3', 'hash-6']}
hash-2 path:{'hash-2': ['hash-2'], 'hash-3': ['hash-2', 'hash-3'], 'hash-1': ['hash-2', 'hash-1'], 'hash-4': ['hash-2', 'hash-3', 'hash-4'], 'hash-5': ['hash-2', 'hash-3', 'hash-5'], 'hash-6': ['hash-2', 'hash-3', 'hash-6']}
hash-3 path:{'hash-3': ['hash-3'], 'hash-2': ['hash-3', 'hash-2'], 'hash-4': ['hash-3', 'hash-4'], 'hash-5': ['hash-3', 'hash-5'], 'hash-6': ['hash-3', 'hash-6'], 'hash-1': ['hash-3', 'hash-2', 'hash-1']}
hash-4 path:{'hash-4': ['hash-4'], 'hash-3': ['hash-4', 'hash-3'], 'hash-2': ['hash-4', 'hash-3', 'hash-2'], 'hash-5': ['hash-4', 'hash-3', 'hash-5'], 'hash-6': ['hash-4', 'hash-

# 測站在河流的順序與距離

In [329]:
#＃所有在頭前溪一公里內測站，在河內的位置
def point_dist(p1,p2):
    #"POINT(262524.23 2726396.01)"
    import math
    xy1 = p1.split("(")[1].split(")")[0].split()
    xy2 = p2.split("(")[1].split(")")[0].split()
    dist = math.pow(math.pow(float(xy1[0])-float(xy2[0]),2)+math.pow(float(xy1[1])-float(xy2[1]),2),0.5)
    return dist

"""
測站： 所有水利署測站，選取水位測站
dist: 測站跟河流的距離
loc: 測站在河流的位置的百分比 0-源頭, 1-海
loc_dist_m: 測站與下一個測站的距離，單位為 m
loc_time_h: 測站與下一個測站用平均流速要多久，單位為 h
height_m: 測站高程
"""

#點位查高程
sql="""select S.type,S.name,ST_Distance(ST_SetSRID(S.geom,3826),ST_Transform(R.wkb_geometry,3826)) as dist, MultiLineLocatePoint(ST_Transform(R.wkb_geometry,3826),ST_SetSRID(S.geom,3826)) as loc, ST_AsEWKT(S.geom) as geom_wkt
from sensor_station as S
join c1300 as R
    on ST_DWithin(ST_SetSRID(S.geom,3826),ST_Transform(R.wkb_geometry,3826),1000)
order by loc
"""
avg_speed = 12000.0 # m/hr
#print("計算用的平均流速為 %.0f m/hr" % (avg_speed))
sql_t="SELECT rid,ST_Value(rast, 'SRID=3826;%s'::geometry) as height \
FROM c1300_dtm_20m \
  WHERE ST_Intersects(rast, 'SRID=3826;%s'::geometry);"

df1 = sql_to_df(conn,sql)
# 過濾掉 竹林大橋(左岸), 因為還有一個同位置測站 竹林大橋
df1=df1[df1['name']!='竹林大橋(左岸)']
#頭前溪的總長度
sql="select ST_Length(ST_Transform(wkb_geometry,3826)) as length from c1300 limit 1"
df2=sql_to_df(conn,sql)
df2.columns
river_length = df2['length'].to_list()[0]
#print("頭前溪河流長度為 %.1f m" %(river_length))
#顯示水位測站之間的距離
df3=df1[df1['type']=='河川水位測站'].sort_values(by=['loc'])
#df4=df3['loc']
#df4.diff()

#計算離下一站多遠
first=True
loc_list=[]
idx_list=[]
time_list=[]
pd_list=[]
curve_rate_list=[]
height_list=[]
slope_list=[]
for index, row in df3.iterrows():
    loc_cur = row['loc']
    pos_cur=row['geom_wkt']
    idx_cur=index

    wkt=row['geom_wkt']
    sql = sql_t % (wkt,wkt)
    #print("sql=%s" %(sql))
    df4 = sql_to_df(conn,sql)
    height_cur = df4['height'].to_list()[0]
    height_list.append(height_cur)

    if not first:
        loc_diff = (loc_cur - loc_prev)* river_length
        loc_list.append(loc_diff)
        idx_list.append(idx_prev)
        time_list.append(loc_diff/avg_speed)
        pd_value=point_dist(pos_cur,pos_prev)
        pd_list.append(pd_value)
        curve_rate_list.append(loc_diff/pd_value)
        slope=loc_diff/(height_prev-height_cur)
        slope_list.append(slope)
        
    loc_prev = loc_cur
    idx_prev= idx_cur
    pos_prev=pos_cur
    height_prev=height_cur
    first=False

station_cnt=len(df3.index)
if station_cnt>=1:
    loc_diff = (1 - loc_prev)* river_length
    loc_list.append(loc_diff)
    idx_list.append(idx_prev)
    time_list.append(loc_diff/avg_speed)
    pd_list.append(loc_diff)
    curve_rate_list.append(1.0)
    slope=loc_diff/(height_cur)
    slope_list.append(slope)
    
#append field
loc_ser = pd.Series(loc_list,index=idx_list)
loc_ser_time = pd.Series(time_list,index=idx_list)
loc_ser_point_dist = pd.Series(pd_list,index=idx_list)
loc_ser_curve_rate = pd.Series(curve_rate_list,index=idx_list)
loc_ser_height = pd.Series(height_list,index=idx_list)
loc_ser_slope = pd.Series(slope_list,index=idx_list)

df3['loc_dist_m']=loc_ser
df3['loc_time_h']=loc_ser_time
df3['point_dist_m']=loc_ser_point_dist
df3['curve_rate']=loc_ser_curve_rate
df3['height_m']=loc_ser_height
df3['slope']=loc_ser_slope
pd.options.display.float_format = '{:,.2f}'.format
print("頭前溪範圍一公里內水位測站\n \
dist: 與河流距離,loc:在頭前溪河流(總長 %.2f m)的位置比例\n \
loc_dist_m:與下一個測站河流距離（m）,loc_time_h: 以平均流速 %.2f m/h, 需要多久(h)\n \
point_dist_m: 與下一個測站直線距離(m),curve_rate: 河流距離/直線距離的比例\n \
height_m: 測站高程(m),slope: 測站間平均坡度=距離/高度差"
      % (river_length,avg_speed))

df3   

頭前溪範圍一公里內水位測站
 dist: 與河流距離,loc:在頭前溪河流(總長 71235.71 m)的位置比例
 loc_dist_m:與下一個測站河流距離（m）,loc_time_h: 以平均流速 12000.00 m/h, 需要多久(h)
 point_dist_m: 與下一個測站直線距離(m),curve_rate: 河流距離/直線距離的比例
 height_m: 測站高程(m),slope: 測站間平均坡=距離/高度差


,type,name,dist,loc,geom_wkt,loc_dist_m,loc_time_h,point_dist_m,curve_rate,height_m,slope
2,河川水位測站,五峰大橋,161.86,0.45,POINT(262524.23 2726396.01),"3,912.76",0.33,"2,891.98",1.35,262.00,78.26
4,河川水位測站,上坪,41.19,0.50,POINT(261568.14 2729125.38),"12,769.63",1.06,"8,957.34",1.43,212.00,116.09
10,河川水位測站,竹林大橋,228.14,0.68,POINT(259465.76 2737832.5),"6,392.21",0.53,"5,598.77",1.14,102.00,152.20
12,河川水位測站,中正大橋,102.83,0.77,POINT(255763 2742032),"4,740.64",0.40,"4,037.23",1.17,60.00,182.33
14,河川水位測站,柯子湖溪匯流站,264.71,0.84,POINT(252144.44 2743822.32),526.47,0.04,698.67,0.75,34.00,87.74
15,河川水位測站,經國橋,132.46,0.85,POINT(251968.79 2744498.55),"8,923.80",0.74,"7,973.18",1.12,28.00,405.63
21,河川水位測站,舊港橋,307.98,0.97,POINT(244886.69 2748161.26),"1,449.10",0.12,"1,345.90",1.08,6.00,289.82
25,河川水位測站,竹港大橋,0.08,0.99,POINT(243658.05 2748710.71),644.76,0.05,644.76,1.00,1.00,644.76
